In [1]:
# General import of torch.
import torch
# Import for graph blocks of torch.
import torch.nn as nn
# Import the models library, to get the model to be used.
import torchvision.models as models
# Import optim library, to get the optimizer to be used.
import torch.optim as optim
# Import torchvision, to manage the input data.
import torchvision
# To apply transformations to the data (when loaded).
import torchvision.transforms as transforms
# Create a Dataset.
from torch.utils.data import Dataset
# To calculate softmax.
from torch.nn import Softmax
softmax = Softmax(dim=1)

# Metrics.
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

# General imports.
import os
import time
import glob
import wandb
import random
import natsort
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from pathlib import Path
from sklearn import manifold
from sklearn.manifold import TSNE

import warnings
warnings.filterwarnings('ignore')

### Enviroment configuration.

In [2]:
# Input size of the model.
inputSize = 'inputSize'
# Output size of the model.
outputSize = 'outputSize'
# Batch size.
batchSize = 'batchSize'
# Epoch init.
epochsInit = 'epochInit'
# Epochs amount.
epochs = 'epochs'
# Class names.
classes = 'classes'
# Class IDs.
classesID = 'classesIDs'
# Number of classes to classify.
classesLen = 'classesLen'
# Group type, for wandb.
groupType = 'groupType'

config = {
    inputSize    : 224,
    outputSize   : 39,
    batchSize    : 32,
    epochsInit   : 1,
    epochs       : 50,
    classes : [
        'Apple - Apple scab',
        'Apple - Black rot',
        'Apple - Cedar apple rust',
        'Apple - Healthy',
        'Background without leaves',
        'Blueberry - Healthy',
        'Cherry - Healthy',
        'Cherry - Powdery mildew',
        'Corn - Cercospora',
        'Corn - Common rust',
        'Corn - Healthy',
        'Corn - Northern Leaf Blight',
        'Grape - Black rot',
        'Grape - Esca',
        'Grape - Healthy',
        'Grape - Leaf blight',
        'Orange - Haunglongbing',
        'Peach - Bacterial spot',
        'Peach - Healthy',
        'Pepper bell - Bacterial spot',
        'Pepper bell - healthy',
        'Potato - Early blight',
        'Potato - Healthy',
        'Potato - Late blight',
        'Raspberry - healthy',
        'Soybean - Healthy',
        'Squash - Powdery mildew',
        'Strawberry - Healthy',
        'Strawberry - Leaf scorch',
        'Tomato - Bacterial spot',
        'Tomato - Early blight',
        'Tomato - Healthy',
        'Tomato - Late blight',
        'Tomato - Leaf Mold',
        'Tomato - Septoria leaf spot',
        'Tomato - Spider mites',
        'Tomato - Target Spot',
        'Tomato - Mosaic virus',
        'Tomato - Yellow Leaf Curl Virus'
    ],
    classesID : [i + 1 for i in range(39)],
    classesLen : 39
}

# Device to be used, prefer cuda, if available.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# About metrics.
# Metric dictionary keys.
# For preprocessing.
_loss        = 'Loss'
# For torch save
_model           = 'Model State Dic'
_optimizer       = 'Optimizer State Dic'
_criterion       = 'Criterion'
_epoch           = 'Epoch'

# Get a clean dictionary for the metrics.
def getMetricsDict():
    return {
        _loss : torch.tensor(0.)
    }

# Function used to update the dictionary of resulting metrics.
def updateRunningMetrics(logits, groundtruth, loss, batchAmount, metricsResults):
    # Accumulate the loss.
    metricsResults[_loss] += loss / batchAmount

# Function used to process the dictionary of resulting metrics (make final calculations).
def processRunningMetrics(metricsResults):
    # Detach the other values in the dictionary.
    metricsResults[_loss] = metricsResults[_loss].detach()

# Pretty print the metrics dictionaries.
def printMetricsDict(metricsResults):
    # All metrics to print
    metricPrints = []

    # Format the loss.
    lossPrint = 'Loss: {:.5f}'.format(metricsResults[_loss])
    metricPrints.append(lossPrint)

    print(', '.join(metricPrints), end='')

# This functions process an metrics result dictionary for wandb. Is necessary to indicte
#   the metrics origin, training or testing.
def processMetricsWandb(metricsResults, training=False):
    # Get the prefix to log on wandb, the keys must be different.
    resultsType = 'training' if training else 'testing'

    # All the wandb keys are based in the original metrics results keys.
    lossKey = '{} ({})'.format(_loss, resultsType)

    # Make the dictionary for wandb and store the values.
    wandbDict = {
        lossKey : metricsResults[_loss].item()
    }

    # Return, to log later.
    return wandbDict

# Get the metrics dictionaries for wandb and log them.
def logMetricsWandb(trainMetricsResults, testMetricsResults):
    # Get both dictionaries for wandb.
    wandbTrainDict = processMetricsWandb(trainMetricsResults, training=True)
    wandbTestDict  = processMetricsWandb(testMetricsResults, training=False)

    # Merge the dictionaries.
    wandbDict = {**wandbTrainDict, **wandbTestDict}

    # Log on wandb
    wandb.log(wandbDict)

# Function used to save the model and the metrics.
def saveEpochData(model, optimizer, criterion, epoch, rootPath):
    # Create a dir for the current epoch.
    Path(rootPath).mkdir(parents=True, exist_ok=True)

    # Path
    savePath = os.path.join(rootPath, 'model.pth')

    # Make dict for torch.save
    saveDict = {
        _model     : model.state_dict(),
        _optimizer : optimizer.state_dict(),
        _criterion : criterion,
        _epoch     : epoch
    }
    
    # Save
    torch.save(saveDict, savePath)

### Loader function.
Should return the training loader and test loader, a iterable object by batches.

In [4]:
class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        self.total_imgs = [f for f in glob.glob(os.path.join(main_dir, "**", "*.jpeg"), recursive=True)]
        #all_imgs = os.listdir(main_dir)
        #self.total_imgs = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        return (tensor_image, tensor_image)

# Transformation definitions.
transformTrain = transforms.Compose([
        transforms.RandomResizedCrop(config[inputSize]),  # This one does a resize (it cuts randomly, it doesn't keep the whole image).
        transforms.RandomHorizontalFlip(),                # Flip the image horizontally randomly.
        transforms.ToTensor()                             # Make the image a tensor.
    ])
transformTest = transforms.Compose([
        transforms.Resize(config[inputSize]),             # Resize the image, keeping all pixels.
        transforms.CenterCrop(config[inputSize]),         # Cut the image in the center.
        transforms.ToTensor()                             # Make the image a tensor.
    ])

# Function used to get the data loaders.
# A folder with two folders inside called train and test is expected as a rootPath.
def getLoaders(trainPath, testPath):

    # Get the training and test data, apply the transformations.
    trainset = CustomDataSet(trainPath, transformTrain)
    testset  = CustomDataSet(testPath,  transformTest)

    # Get the loaders, to iterate the data through batches.
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=config[batchSize], shuffle=True, num_workers=2)
    testloader  = torch.utils.data.DataLoader(testset,  batch_size=config[batchSize], shuffle=False, num_workers=2)
    
    return trainloader, testloader

# Network definition

### Function used to create a coder.

In [5]:
def _create_coder(channels, kernel_sizes, strides, conv_types, activation_types, paddings=(0,0), batch_norms=False):
    '''
    Function that creates en- or decoders based on parameters
    Args:
        channels ([int]): Channel sizes per layer. 1 more than layers
        kernel_sizes ([int]): Kernel sizes per layer
        strides ([int]): Strides per layer
        conv_types ([f()->type]): Type of the convoultion module per layer
        activation_types ([f()->type]): Type of activation function per layer
        paddings ([(int, int)]): The padding per layer
        batch_norms ([bool]): Whether to use batchnorm on each layer
    Returns (nn.Sequential): The created coder
    '''
    if not isinstance(conv_types, list):
        conv_types = [conv_types for _ in range(len(kernel_sizes))]

    if not isinstance(activation_types, list):
        activation_types = [activation_types for _ in range(len(kernel_sizes))]

    if not isinstance(paddings, list):
        paddings = [paddings for _ in range(len(kernel_sizes))]
        
    if not isinstance(batch_norms, list):
        batch_norms = [batch_norms for _ in range(len(kernel_sizes))]

    coder = nn.Sequential()
    for layer in range(len(channels)-1):
        coder.add_module(
            'conv'+ str(layer), 
            conv_types[layer](
                in_channels=channels[layer], 
                out_channels=channels[layer+1],
                kernel_size=kernel_sizes[layer],
                stride=strides[layer]
            )
        )
        if batch_norms[layer]:
            coder.add_module(
                'norm'+str(layer),
                nn.BatchNorm2d(channels[layer+1])
            )
        if not activation_types[layer] is None:
            coder.add_module('acti'+str(layer),activation_types[layer]())

    return coder


### VAE definition

In [6]:
class TemplateVAE(nn.Module):
    '''
    A template class for Variational Autoencoders to minimize code duplication
    Args:
        input_size (int,int): The height and width of the input image
        z_dimensions (int): The number of latent dimensions in the encoding
        variational (bool): Whether the model is variational or not
        gamma (float): The weight of the KLD loss
        perceptual_net: Which perceptual network to use (None for pixel-wise)
    '''
    
    def encode(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0),-1)
        mu = self.mu(x)
        logvar = self.logvar(x)
        return mu, logvar

    def sample(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        eps = torch.autograd.Variable(std.data.new(std.size()).normal_())
        out = eps.mul(std).add_(mu)
        return out

    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        if self.variational:
            z = self.sample(mu, logvar)
        else:
            z = mu
        rec_x = self.decode(z)
        return rec_x, z, mu, logvar

    def loss(self, output, x):
        rec_x, z, mu, logvar = output
        if self.perceptual_loss:
            x = self.perceptual_net(x)
            rec_x = self.perceptual_net(rec_x)
        else:
            x = x.reshape(x.size(0), -1)
            rec_x = rec_x.view(x.size(0), -1)
            
        REC = nn.functional.mse_loss(rec_x, x, reduction='mean')

        if self.variational:
            KLD = -1 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
            return REC + self.gamma*KLD, REC, KLD
        else:
            return [REC]

class FourLayerCVAE(TemplateVAE):
    '''
    A Convolutional Variational Autoencoder for images
    Args:
        input_size (int,int): The height and width of the input image
            acceptable sizes are 64+16*n
        z_dimensions (int): The number of latent dimensions in the encoding
        variational (bool): Whether the model is variational or not
        gamma (float): The weight of the KLD loss
        perceptual_net: Which perceptual network to use (None for pixel-wise)
    '''

    def __init__(self, input_size=(64,64), z_dimensions=32,
        variational=True, gamma=20.0, perceptual_net=None
    ):
        super().__init__()

        #Parameter check
        if (input_size[0] - 64) % 16 != 0 or (input_size[1] - 64) % 16 != 0:
            raise ValueError(
                f'Input_size is {input_size}, but must be 64+16*N'
            )

        #Attributes
        self.input_size = input_size
        self.z_dimensions = z_dimensions
        self.variational = variational
        self.gamma = gamma
        self.perceptual_net = perceptual_net

        self.perceptual_loss = not perceptual_net is None
            
        encoder_channels = [3,32,64,128,256]
        self.encoder = _create_coder(
            encoder_channels, [4,4,4,4], [2,2,2,2],
            nn.Conv2d, nn.ReLU,
            batch_norms=[True,True,True,True]
        )
        
        f = lambda x: np.floor((x - (2,2))/2)
        conv_sizes = f(f(f(f(np.array(input_size)))))
        conv_flat_size = int(encoder_channels[-1]*conv_sizes[0]*conv_sizes[1])
        self.mu = nn.Linear(conv_flat_size, self.z_dimensions)
        self.logvar = nn.Linear(conv_flat_size, self.z_dimensions)

        g = lambda x: int((x-64)/16)+1
        deconv_flat_size = g(input_size[0]) * g(input_size[1]) * 1024
        self.dense = nn.Linear(self.z_dimensions, deconv_flat_size)

        self.decoder = _create_coder(
            [1024,128,64,32,3], [5,5,6,6], [2,2,2,2],
            nn.ConvTranspose2d,
            [nn.ReLU,nn.ReLU,nn.ReLU,nn.Sigmoid],
            batch_norms=[True,True,True,False]
        )

        self.relu = nn.ReLU()

    def decode(self, z):
        y = self.dense(z)
        y = self.relu(y)
        y = y.view(
            y.size(0), 1024,
            int((self.input_size[0]-64)/16)+1,
            int((self.input_size[1]-64)/16)+1
        )
        y = self.decoder(y)
        return y

### Perceptual net creation.

In [7]:
architecture_features = {
    'alexnet' : ['features'],
    'vgg16' : ['features'],
}

class SimpleExtractor(nn.Module):
    '''
    A simple feature extractor for torchvision models
    Args:
        architecture (str): The architecture to extract from
        layer (int): The sub-module in 'features' to extract at
        frozen (bool): Whether the network can be trained
        sigmoid_out (bool): Whether to normalize the output with a sigmoid
    '''
    def __init__(self, architecture, layer, frozen=True, sigmoid_out=True):
        super(SimpleExtractor, self).__init__()
        self.architecture = architecture
        self.layer = layer
        self.frozen = frozen
        self.sigmoid_out = sigmoid_out
    
        os.environ['TORCH_HOME'] = './'
        original_model = models.__dict__[architecture](pretrained=True)
        original_features = original_model
        for attribute in architecture_features[architecture]:
            original_features = getattr(original_features, attribute)
        self.features = nn.Sequential(
            *list(original_features.children())[:layer]
        )
        if sigmoid_out:
            self.features.add_module('sigmoid',nn.Sigmoid())
        if frozen:
            self.eval()
            for param in self.features.parameters():
                param.requires_grad = False

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return x

def AlexNet(layer=5, pretrained=True, frozen=True, sigmoid_out=True):
    return SimpleExtractor('alexnet', layer, frozen, sigmoid_out)

def VGG16(layer=5, pretrained=True, frozen=True, sigmoid_out=True):
    return SimpleExtractor('vgg16', layer, frozen, sigmoid_out)

### Epoch progress

In [8]:
def epochProgress(currentBatch, totalBatch, loss, time, type):
    # Base string.
    line = '\r{} - [{}/{}] - Losses: {}, Time elapsed: {}s'
    # Time string.
    elapsedTime = '{0:.1f}'.format(time)
    # Loss.
    actualLoss = '{0:.5f}'.format(loss)
    print(line.format(type, currentBatch, totalBatch, actualLoss, elapsedTime), end='')

### Training method.
This method takes care of a single training pass. Another function call this one multiple times.

In [9]:
def trainEpoch(dataloader, model, criterion, optimizer):
    # Get init time.
    start_time = time.time()

    # Metrics for training.
    metricsResults = getMetricsDict()

    # Enable the grad, for training.
    with torch.set_grad_enabled(True):

        # Indicate that the model is going to be trained.
        model.train()

        # Loader len, for metrics calculation.
        loaderLen = len(dataloader)

        # Iterate the batches for training.
        for i, batch in enumerate(dataloader):

            # Train the model.
            # Get the inputs and labels, and move them to the selected device.
            inputs, labels = batch[0].to(device), batch[1].to(device)

            # Zero the gradient parameters.
            optimizer.zero_grad()
            # Get the predictions.
            outputs = model(inputs)
            # Calculate the error.
            loss = criterion(outputs, labels)
            # Calculates the derivatives of the parameters that have a gradient.
            loss[0].backward()
            # Update the parameters based on the computer gradient.
            optimizer.step()
            # Metrics for the training set.
            updateRunningMetrics(outputs, labels, loss[0].item(), loaderLen, metricsResults)

            epochProgress(i + 1, loaderLen, loss[0].item(), time.time() - start_time, 'Training')

    return metricsResults

### Evaluation method.
This method evaluates the model for a specified dataset.

In [10]:
def evaluate(dataloader, model, criterion):
    # Get init time.
    start_time = time.time()
    
    # Metrics for testing.
    metricsResults = getMetricsDict()

    # Enable the grad, for training.
    with torch.set_grad_enabled(False):

        # Indicate that the model is going to be evaluated.
        model.eval()

        # Loader len, for metrics calculation.
        loaderLen = len(dataloader)

        # Iterate the batches for testing.
        for i, batch in enumerate(dataloader):
            # Test the model.
            # Get the inputs and labels, and move them to the selected device.
            inputs, labels = batch[0].to(device), batch[1].to(device)
            # Get the predictions.
            outputs = model(inputs)
            # Calculate the error.
            loss = criterion(outputs, labels)
            # Metrics for the testing set.
            updateRunningMetrics(outputs, labels, loss[0].item(), loaderLen, metricsResults)

            epochProgress(i, loaderLen, loss[0].item(), time.time() - start_time, 'Evaluating')

    return metricsResults

### Display method.

In [11]:
def showPredictions(loader, model, folder, epoch, saveOriginal):
    # Enable the grad, for training.
    with torch.set_grad_enabled(False):

        # Indicate that the model is going to be evaluated.
        model.eval()
        
        # Get a batch from the loader.
        firstBatch = next(iter(loader))

        # Get the inputs and labels, and move them to the selected device.
        inputs, labels = firstBatch[0].to(device), firstBatch[1].to(device)
        # Get the predictions.
        rec_x, z, mu, logvar = model(inputs)

        torchvision.utils.save_image(rec_x, f"{folder}/{epoch}_pred.png")
        if saveOriginal:
            torchvision.utils.save_image(labels, f"{folder}/{epoch}_orig.png")
            

### Trainining and evaluate method.
For the specific purpose of this project, in each epoch we evaluate metrics for each data set (training and testing) in each epoch, this method simplifies the process. 

In [12]:
def trainAndEvaluate(trainloader, testloader, model, criterion, optimizer, display, savePath):

    originalDisplayed = False

    for epoch in range(config[epochsInit], config[epochs] + 1):
        
        # Train the model.
        trainMetricsResults = trainEpoch(trainloader, model, criterion, optimizer)
        processRunningMetrics(trainMetricsResults)

        # Evaluate the model.
        testMetricsResults = evaluate(testloader, model, criterion)
        processRunningMetrics(testMetricsResults)

        # Display some random images.
        if display is not None:
            showPredictions(trainloader, model, display, epoch, not originalDisplayed)
            #originalDisplayed = True

        # Log on wandb
        logMetricsWandb(trainMetricsResults, testMetricsResults)

        # Display latent vectors in images.
        if epoch % 5 == 0:
            runTSNE(model, epoch, folder="tsneImages/")

        # Save model and metrics for the epochs.
        saveEpochData(model, optimizer, criterion, epoch, savePath)

        # Print the results.
        if epoch % 1 == 0:
            print('\r**', '[', 'Epoch ', epoch, ']', '*' * 48, sep='')
            print('\tTraining', end=' ')
            printMetricsDict(trainMetricsResults)
            print(end=' | ')
            print('Evaluation', end=' ')
            printMetricsDict(testMetricsResults)
            print()

### Function used to visualize the latent vector with TSNE

In [13]:
def saveTSNE(loader, model, epoch, folder):
    # Get init time.
    start_time = time.time()
    
    # Enable the grad, for training.
    with torch.set_grad_enabled(False):

        # Indicate that the model is going to be evaluated.
        model.eval()

        pred_acc = torch.tensor(()).to(device)
        labels_acc = torch.tensor(()).to(device)

        top = len(loader)

        for i, (images, labels) in enumerate(loader):
            images = images.to(device)
            labels = labels.to(device)

            rec_x, z, mu, logvar = model(images)

            #print(z.size())
            #input()

            pred_acc = torch.cat((pred_acc, z), 0)
            labels_acc = torch.cat((labels_acc, labels), 0)

            if i == top:
                break

            epochProgress(i + 1, top, 0, time.time() - start_time, 'TSNE')

        print(end=' fitting TSNE,')
        tsne = TSNE(n_components=3, verbose=0, perplexity=40, n_iter=1500)

        pred_acc = pred_acc.to("cpu").detach().numpy()
        Y = tsne.fit_transform(pred_acc)
        labels_acc = labels_acc.to("cpu")
        print(end=' done,')

        fig = plt.figure(figsize=(12.8*1.5, 9.6*1.5))
        ax = fig.add_subplot(projection='3d')

        print(end=' plotting image.')
        ax.scatter(Y[:, 0], Y[:, 1], Y[:, 2], c=labels_acc)

        plt.savefig(f"{folder}/epoch_TSNE_epoch_{epoch}.png")

def runTSNE(model, epoch, folder="tsneImages/"):
    dataPathTest = '/home/pablo/Desktop/ML/Proyectos/Proyecto III/data/corrida1/labelTest'

    transformTest = transforms.Compose([
            transforms.Resize(config[inputSize]),             # Resize the image, keeping all pixels.
            transforms.CenterCrop(config[inputSize]),         # Cut the image in the center.
            transforms.ToTensor()                             # Make the image a tensor.
        ])

    testset  = torchvision.datasets.ImageFolder(dataPathTest, transformTest)

    # Get the loaders, to iterate the data through batches.
    testloader = torch.utils.data.DataLoader(testset, batch_size=config[batchSize], shuffle=True, num_workers=2)

    saveTSNE(testloader, model, epoch, folder)

In [14]:
def loadCheckpoint(model, optimizer, criterion, loadPath):
    # Load the checkpoint.
    checkpoint = torch.load(loadPath)
    # Load model.
    model.load_state_dict(checkpoint[_model])
    # Load optimizer.
    optimizer.load_state_dict(checkpoint[_optimizer])
    # Load criterion.
    criterion = checkpoint[_criterion]
    # Set epoch.
    config[epochsInit] = checkpoint[_epoch] + 1

    return model, optimizer, criterion

def executeTest(network, dataPath, runName, savePath, display=None, loadPath=None):
    # Get the model.
    # Get a predefined model from pytorch, without the pretrained parameters.
    net = network

    # Load the model to the selected device.
    net.to(device)
    
    # Get criterion and optimizer.
    # Optimizer and the loss funtion used to train the model.
    criterion = network.loss
    optimizer = optim.Adam(net.parameters())

    # Check if required to load a checkpoint.
    if loadPath != None:
        net, optimizer, criterion = loadCheckpoint(net, optimizer, criterion, loadPath)
        net.to(device)

    # Get the loaders.
    trainloader, testloader = getLoaders(dataPath[0], dataPath[1])

    # Init wandb
    run = wandb.init(project='VAE', entity='tecai', config=config, name=runName)

    # Train and evaluate
    trainAndEvaluate(trainloader, testloader, net, criterion, optimizer, display, savePath)

    # Finish wandb
    run.finish()

In [15]:
# Paths of data.
dataPathTrain = '/home/pablo/Desktop/ML/Proyectos/Proyecto III/data/corrida1/noLabel'
dataPathTest = '/home/pablo/Desktop/ML/Proyectos/Proyecto III/data/corrida1/labelTest'
displayPath = './predictedImages'

savePath = '/media/pablo/Disquito/IA/VAE/firstest'

lossNetowrk = perceptual_net=VGG16(5) 
# Network.
network = FourLayerCVAE(input_size=(config[inputSize], config[inputSize]), z_dimensions=1024, variational=True, gamma=0.001, perceptual_net=lossNetowrk)

# Execute.
executeTest(network, (dataPathTrain, dataPathTest), None, savePath, display=displayPath)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pablobrenes (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


**[Epoch 1]************************************************
	Training Loss: 0.00543 | Evaluation Loss: 0.00520
**[Epoch 2]************************************************
	Training Loss: 0.00485 | Evaluation Loss: 0.00495
**[Epoch 3]************************************************
	Training Loss: 0.00452 | Evaluation Loss: 0.00448
**[Epoch 4]************************************************
	Training Loss: 0.00421 | Evaluation Loss: 0.00548
**[Epoch 5]************************************************
	Training Loss: 0.00410 | Evaluation Loss: 0.00432
**[Epoch 6]************************************************
	Training Loss: 0.00401 | Evaluation Loss: 0.00410
**[Epoch 7]************************************************
	Training Loss: 0.00398 | Evaluation Loss: 0.00397
**[Epoch 8]************************************************
	Training Loss: 0.00396 | Evaluation Loss: 0.00391
**[Epoch 9]************************************************
	Training Loss: 0.00392 | Evaluation Loss: 0.00398
*

Loss (training),0.00361
Loss (testing),0.00353
_runtime,2636
_timestamp,1624219775
_step,49


Loss (training),█▆▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁
Loss (testing),▇▆▄█▃▃▂▃▃▃▂▅▂▂▄▂▂▂▂▃▂▂▂▂▂▁▂▁▂▁▁▁▂▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
